In [1]:
# 导入工具库
import os
import numpy as np
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision

2024-05-18 20:39:28.948782: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
# 读取X射线图像
def imageread(path,width=512,height=512):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (width, height))
    x = x/255.0
    x = x.astype(np.float32)
    return x
# 读取标签蒙版（mask，即和图片同样大小的标签）
def maskread(path,width=512,height=512):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (width, height))
    x = x/np.max(x)
    x = x > 0.5
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x

In [3]:
"""加载与切分数据"""
def load_data(path, split=0.1):
    images = sorted(glob(os.path.join(path, "CXR_png", "*.png")))
    masks = sorted(glob(os.path.join(path, "Mask", "*.png")))
    split_size = int(len(images) * split) # 9:1的比例切分
    train_x, val_x = train_test_split(images, test_size=split_size, random_state=42)
    train_y, val_y = train_test_split(masks, test_size=split_size, random_state=42)
    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=split_size, random_state=42)
    return (train_x, train_y), (val_x, val_y), (test_x, test_y)

In [4]:
# tensor格式转换
def tf_parse(x, y):
    def _parse(x, y):
        x = x.decode()
        y = y.decode()
        x = imageread(x)
        y = maskread(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([512, 512, 3])
    y.set_shape([512, 512, 1])
    return x, y


# 构建tensorflow dataset
def tf_dataset(X, Y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.shuffle(buffer_size=200)
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(4)
    return dataset


In [5]:

# U-Net 网络构建
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, \
    Input
from tensorflow.keras.models import Model


# 一个卷积块结构
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x


# 编码器模块
def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p


# 解码器模块
def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x


# 完整的U-Net
def build_unet(input_shape):
    inputs = Input(input_shape)
    # 编码器部分
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)
    b1 = conv_block(p4, 1024)
    # 解码器部分
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)
    # 输出
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)
    model = Model(inputs, outputs, name="U-Net")
    return model


In [6]:
# 评估准则与损失函数
# IoU计算
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)
# Dice Loss定义
smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)
def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [7]:
# 超参数设置与模型编译
# 超参数
batch_size = 2
lr = 1e-5
epochs = 30
model_path = "models/model.h5"
# 读取数据
dataset_path = './data'
(train_x, train_y), (val_x, val_y), (test_x, test_y) = load_data(dataset_path)
# 训练集与验证集
train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
val_dataset = tf_dataset(val_x, val_y, batch=batch_size)
# 构建模型
model = build_unet((512, 512, 3))
# 评估准则
metrics = [dice_coef, iou, Recall(), Precision()]
# 编译模型
model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=metrics)

2024-05-18 20:39:30.114602: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-05-18 20:39:30.162293: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:3e:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.75GiB deviceMemoryBandwidth: 573.69GiB/s
2024-05-18 20:39:30.162348: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2024-05-18 20:39:30.172939: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2024-05-18 20:39:30.173062: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2024-05-18 20:39:30.176331: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library li

In [8]:
# 回调函数
callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-8, verbose=1)
        ]
# 模型训练
history = model.fit(
        train_dataset,
        epochs=epochs,
        validation_data=val_dataset,
        callbacks=callbacks
    )

2024-05-18 20:39:31.725786: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2024-05-18 20:39:31.745198: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2500000000 Hz


Epoch 1/30


2024-05-18 20:39:34.889399: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2024-05-18 20:39:35.761049: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8101
2024-05-18 20:39:36.870280: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2024-05-18 20:39:37.545364: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


56/56 [==============================] - 54s 799ms/step - loss: 0.5351 - dice_coef: 0.4649 - iou: 0.3075 - recall: 0.7166 - precision: 0.5603 - val_loss: 0.6624 - val_dice_coef: 0.3450 - val_iou: 0.2097 - val_recall: 0.6469 - val_precision: 0.3378

Epoch 00001: val_loss improved from inf to 0.66243, saving model to models/model.h5
Epoch 2/30
56/56 [==============================] - 47s 844ms/step - loss: 0.3450 - dice_coef: 0.6550 - iou: 0.4925 - recall: 0.9077 - precision: 0.7013 - val_loss: 0.6643 - val_dice_coef: 0.3374 - val_iou: 0.2036 - val_recall: 0.4189 - val_precision: 0.2937

Epoch 00002: val_loss did not improve from 0.66243
Epoch 3/30
56/56 [==============================] - 47s 838ms/step - loss: 0.2171 - dice_coef: 0.7829 - iou: 0.6473 - recall: 0.9556 - precision: 0.8985 - val_loss: 0.6788 - val_dice_coef: 0.3262 - val_iou: 0.1962 - val_recall: 0.1362 - val_precision: 0.1580

Epoch 00003: val_loss did not improve from 0.66243
Epoch 4/30
56/56 [===========================

In [11]:
# 重新载入模型
from tensorflow.keras.utils import CustomObjectScope
with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
  model = tf.keras.models.load_model("./models/model.h5")
# 测试集预估
from tqdm import tqdm
import matplotlib.pyplot as plt
ct=0
# 遍历测试集
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracing the image name. """
    image_name = x.split("/")[-1]
    # 读取测试图片集
    ori_x = cv2.imread(x, cv2.IMREAD_COLOR)
    ori_x = cv2.resize(ori_x, (512, 512))
    x = ori_x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)
    # 读取标签信息
    ori_y = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    ori_y = cv2.resize(ori_y, (512, 512))
    ori_y = np.expand_dims(ori_y, axis=-1)  ## (512, 512, 1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)  ## (512, 512, 3)
    # 预估
    y_pred = model.predict(x)[0] > 0.5
    y_pred = y_pred.astype(np.int32)
    #plt.imshow(y_pred)
    # 存储预估结果mask
    save_image_path = "./"+str(ct)+".png"
    ct+=1
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)
    sep_line = np.ones((512, 10, 3)) * 255
    cat_image = np.concatenate([ori_x, sep_line, ori_y, sep_line, y_pred*255], axis=1)
    cv2.imwrite(save_image_path, cat_image)

100%|██████████| 13/13 [00:06<00:00,  1.96it/s]
